In [86]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")


In [87]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("sample data/input data/pet.csv")
df.head()

,respond_id,pet,count
0,1,Duck,6
1,2,Snake,9
2,3,Samoyed,1
3,4,Dog,6
4,5,British Shorthair,8


In [88]:

# Get unique transactions in the Name / Description column
unique_pet = df["pet"].unique()
len(unique_pet)

23

In [89]:
unique_pet[1:10]

array(['Snake', 'Samoyed', 'Dog', 'British Shorthair', 'Fish', 'Bunny',
       'Golden Retriever', 'Guinea Pig', 'American Shorthair'],
      dtype=object)

In [90]:

# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_pet), 30))
index_list

[0, 23]

In [91]:
def categorize_pet(breed, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: british long hair - Cat, golden retriever - Dog, etc.. Categories should be less than 4 words. " + breed)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'pet vs category': response})
    categories_df[['pet', 'Category']] = categories_df['pet vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [93]:
categorize_pet('hens, goldfish, british longhair',
                        llm)

['1. Hens - Chicken', '', '2. Goldfish - Fish (Aquatic)', '', '3. British Longhair - Cat (Pet)']


,pet vs category,pet,Category
0,1. Hens - Chicken,1. Hens,Chicken
1,,,None
2,2. Goldfish - Fish (Aquatic),2. Goldfish,Fish (Aquatic)
3,,,None
4,3. British Longhair - Cat (Pet),3. British Longhair,Cat (Pet)


In [96]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    breed = unique_pet[index_list[i]:index_list[i+1]]
    breed = ','.join(breed)

    categories_df = categorize_pet(breed, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)


['1. Duck - Waterfowl', '', '2. Snake - Reptile', '', '3. Samoyed - Dog', '', '4. British Shorthair - Cat', '', '5. Fish - Aquatic Life', '', '6. Bunny - Rabbit', '', '7. Golden Retriever - Dog', '', '8. Guinea Pig - Rodent', '', '9. American Shorthair - Cat', '', '10. Parrot - Bird', '', '11. Cat - Felines', '', '12. Chicken - Fowl', '', '13. Turtle - Reptile', '', '14. Bird - Aviary', '', '15. Hens - Fowl', '', '16. Goldfish - Aquatic Life', '', '17. White Shepherd - Dog', '', '18. Rat - Rodent', '', '19. Rabbit - Lagomorpha', '', '20. Horse - Equidae', '', '21. Kitten - Cat', '', '22. Chihuahua - Dog', '', 'These categories should provide a quick and easy way to identify the type of animal or pet associated with each expense.']


In [97]:
categories_df_all


,pet vs category,pet,Category
0,1. Duck - Waterfowl,1. Duck,Waterfowl
1,,,None
2,2. Snake - Reptile,2. Snake,Reptile
3,,,None
4,3. Samoyed - Dog,3. Samoyed,Dog
5,,,None
6,4. British Shorthair - Cat,4. British Shorthair,Cat
7,,,None
8,5. Fish - Aquatic Life,5. Fish,Aquatic Life
9,,,None


In [98]:

# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array(['Waterfowl', None, 'Reptile', 'Dog', 'Cat', 'Aquatic Life',
       'Rabbit', 'Rodent', 'Bird', 'Felines', 'Fowl', 'Aviary',
       'Lagomorpha', 'Equidae'], dtype=object)

In [99]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Feline"), 'Category'] = "Cat"


In [111]:

# Remove the numbering eg "1. " from Transaction column
categories_df_all['pet'] = categories_df_all['pet'].str.replace(r'^\d+\.\s+', '', regex=True)
dog_row = pd.DataFrame({
    'pet vs category': ['23. Dog - Dog'],
    'pet': ['Dog'],
    'Category': ['Dog']
})
categories_df_all = pd.concat([categories_df_all, dog_row], ignore_index=True)
categories_df_all

C:\Users\13930\AppData\Local\Temp\ipykernel_19012\2717830005.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['pet'] = categories_df_all['pet'].str.replace(r'^\d+\.\s+', '', regex=True)


,pet vs category,pet,Category
0,1. Duck - Waterfowl,Duck,Waterfowl
1,2. Snake - Reptile,Snake,Reptile
2,3. Samoyed - Dog,Samoyed,Dog
3,4. British Shorthair - Cat,British Shorthair,Cat
4,5. Fish - Aquatic Life,Fish,Aquatic Life
5,6. Bunny - Rabbit,Bunny,Rabbit
6,7. Golden Retriever - Dog,Golden Retriever,Dog
7,8. Guinea Pig - Rodent,Guinea Pig,Rodent
8,9. American Shorthair - Cat,American Shorthair,Cat
9,10. Parrot - Bird,Parrot,Bird


In [ ]:

# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("sample data/input data/pet.csv")
#df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df_merged = pd.merge(df, categories_df_all, left_on='pet', right_on='pet', how='left')
df_merged

,respond_id,pet,count,pet vs category,Category
0,1,Duck,6,1. Duck - Waterfowl,Waterfowl
1,2,Snake,9,2. Snake - Reptile,Reptile
2,3,Samoyed,1,3. Samoyed - Dog,Dog
3,4,Dog,6,23. Dog - Dog,Dog
4,5,British Shorthair,8,4. British Shorthair - Cat,Cat
...,...,...,...,...,...
195,196,Guinea Pig,1,8. Guinea Pig - Rodent,Rodent
196,197,Goldfish,3,16. Goldfish - Aquatic Life,Aquatic Life
197,198,Chihuahua,5,22. Chihuahua - Dog,Dog
198,199,Snake,7,2. Snake - Reptile,Reptile


In [114]:
df_merged.to_csv("sample data/output data/categorized_pet.csv", index=False)